In [2]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2 #required for Colab

import IPython

IPython.Application.instance().kernel.do_shutdown(restart=True)


[notice] A new release of pip available: 22.2.1 -> 23.3.2
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.10/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Initializing the ZenML global configuration version to 0.55.1
⠇ Installing integrations.....
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7

[notice] A new release of pip available: 22.2.1 -> 23.3.2
[notice] To update, run: /Library/Frameworks/Python.framework/Versions/3.10/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


{'status': 'ok', 'restart': True}

In [1]:
!rm -rf .zen
!zenml init

Creating database tables
⠋ Initializing ZenML repository at /Users/williamliu/Development/mlops.
⠙ Initializing ZenML repository at /Users/williamliu/Development/mlops.
Creating default workspace 'default' ...
Creating default stack in workspace default...
⠹ Initializing ZenML repository at /Users/williamliu/Development/mlops.
Creating default user 'default' ...
⠸ Initializing ZenML repository at /Users/williamliu/Development/mlops.
⠼ Initializing ZenML repository at /Users/williamliu/Development/mlops.
⠴ Initializing ZenML repository at /Users/williamliu/Development/mlops.
⠦ Initializing ZenML repository at /Users/williamliu/Development/mlops.
⠧ Initializing ZenML repository at /Users/williamliu/Development/mlops.
Setting the global active workspace to 'default'.
Setting the global active stack to default.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
⠇ Initializing ZenML repository at /Users/williamliu/Development/mlops.
⠋ Initializing ZenM

In [4]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    x_train, x_test, y_train, y_test = train_test_split(
        data, digits.target, test_size = 0.2, shuffle = False
    )
    model = SVC(gamma = 0.001)
    model.fit(x_train, y_train)
    test_acc = model.score(x_test, y_test)
    print(f"test accuracy: {test_acc}")

In [5]:
train_test()

test accuracy: 0.9583333333333334


In [6]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "x_train"],
    Annotated[np.ndarray, "x_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
    """ Load the digits dataset as numpy arrays """
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    x_train, x_test, y_train, y_test = train_test_split(
        data, digits.target, test_size = 0.2, shuffle = False
    )
    return x_train, x_test, y_train, y_test
    

In [7]:
@step
def SVC_trainer(
    x_train:np.ndarray,
    y_train:np.ndarray
)->ClassifierMixin:
    """ Train a sklearn SVC classifier """
    model = SVC(gamma = 0.001)
    model.fit(x_train, y_train)
    return model
    

In [8]:
@step
def evaluator(
    x_test:np.ndarray,
    y_test:np.ndarray,
    model:ClassifierMixin,
) -> float:
    """ Calculate the test set accuracy of sklearn model """
    test_acc = model.score(x_test, y_test)
    print(f"Test Accuracy: {test_acc}")
    return test_acc

In [9]:
from zenml import pipeline

@pipeline
def digits_pipeline():
    x_train, x_test, y_train, y_test = importer()
    model = SVC_trainer(x_train=x_train, y_train=y_train)
    evaluator(x_test=x_test, y_test=y_test, model=model)

In [10]:
digits_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
Step importer has started.
Step importer has finished in 1.087s.
Step SVC_trainer has started.
Step SVC_trainer has finished in 0.151s.
Step evaluator has started.
Test Accuracy: 0.9583333333333334
Step evaluator has finished in 0.118s.
Pipeline run has finished in 1.515s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


In [11]:
from zenml.environment import Environment

def start_zen_ml_dash(port=8237):
    !zenml up --port {port}

start_zen_ml_dash()

Deploying a local ZenML server with name 'local'.
⠋ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠙ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠹ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠸ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠼ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠴ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠦ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠧ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠇ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠏ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠋ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠙ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠹ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠸ Starting service 'LocalZe